<a href="https://colab.research.google.com/github/whs1111/bert-version/blob/master/src/CLASS/LSTM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 新段落

In [2]:
cd gdrive/My Drive/Colab_Notebooks/code


/content/gdrive/My Drive/Colab_Notebooks/code


In [3]:
#@title
!git clone https://github.com/NVIDIA/apex.git
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
%cd ..

import torch
import apex
from fastai.text import *
import datetime
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

LOG_PATH=Path('logs/')  
MODEL_PATH=Path('models/') 

if not LOG_PATH.exists():
  LOG_PATH.mkdir()
import logging

args = {
    "run_text": "my_test",
    "max_seq_length": 30,
    "do_lower_case": True,
    "train_batch_size": 16,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

device = torch.device('cuda')

if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

fatal: destination path 'apex' already exists and is not an empty directory.
/content/gdrive/My Drive/Colab_Notebooks/code/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-nh1nrefx
Created temporary directory: /tmp/pip-req-tracker-xtwah0z8
Created requirements tracker '/tmp/pip-req-tracker-xtwah0z8'
Created temporary directory: /tmp/pip-install-sgv7wdi1
Processing /content/gdrive/My Drive/Colab_Notebooks/code/apex
  Created temporary directory: /tmp/pip-req-build-tce34ahc
  Added file:///content/gdrive/My%20Drive/Colab_Notebooks/code/apex to build tracker '/tmp/pip-req-tracker-xtwah0z8'
    Running setup.py (path:/tmp/pip-req-build-tce34ahc/setup.py) egg_info for package from file:///content/gdrive/My%20Drive/Colab_Notebooks/code/apex
 

In [4]:
pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 20.2MB/s 
     |████████████████████████████████| 133kB 44.0MB/s 
     |████████████████████████████████| 7.2MB 50.2MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
ERROR: botocore 1.19.42 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [5]:
print(torch.cuda.is_available())

True


In [11]:
from pytorch_pretrained_bert import BertTokenizer,BertModel
import torch
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
a = "i am a dog"
a_tokens = bert_tokenizer.tokenize(a)
print(a_tokens)
a_seq_ids = bert_tokenizer.convert_tokens_to_ids(a_tokens)
print(a_seq_ids)
bert_model = BertModel.from_pretrained("bert-base-uncased").to("cuda")
batch_data = torch.Tensor(a_seq_ids).cuda().long().view((1,-1))
out,_ = bert_model(batch_data)
print(out[0].shape)
print(out[0][0][0])

12/23/2020 05:27:43 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
['i', 'am', 'a', 'dog']
[1045, 2572, 1037, 3899]
12/23/2020 05:27:44 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
12/23/2020 05:27:44 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir 

In [14]:
from pytorch_pretrained_bert import BertTokenizer,BertModel
import torch
bert_model = BertModel.from_pretrained("bert-base-uncased").to("cuda")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
import numpy as np
import csv
label_list = ["GoodsServices",
        "SearchAndRescue",
        "InformationWanted",
        "Volunteer",
        "FundRaising",
        "Donations",
        "MovePeople",
        "FirstPartyObservation",
        "ThirdPartyObservation",
        "Weather",
        "EmergingThreats",
        "NewSubEvent",
        "MultimediaShare",
        "ServiceAvailable",
        "Factoid",
        "Official",
        "CleanUp",
        "Hashtags",
        "ContextualInformation",
        "News",
        "Advice",
        "Sentiment",
        "Discussion",
        "Irrelevant",
        "OriginalEvent"]
important_list = ["Low",
        "Medium",
        "High",
        "Critical"
       ]
tweets = []
path = '/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/attention/data/ll.csv'
with open(path, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for line in reader:
        tweet_full = line
        tweets.append({
            'id': tweet_full[0],
            'label':tweet_full[1],
            'important':tweet_full[2],
            'text': tweet_full[3].lower(),
            # 'name': tweet_full['user']['name'].split()[0]
            })
tweet_list = []
text_list = []
label_list = []
for i in range(len(tweets)):
        #text_list.append(tweets[i]['text'])
        a_tokens = bert_tokenizer.tokenize(tweets[i]['text'])
        a_seq_ids = bert_tokenizer.convert_tokens_to_ids(a_tokens)
        batch_data = torch.Tensor(a_seq_ids).cuda().long().view((1,-1))
        out,_ = bert_model(batch_data)
        print(out[0].shape)
        tweets_vector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        tweets_label = eval(tweets[i]['label'])
        p = 0
        for j in label_list:
            for k in tweets_label:
                if k == j:
                    tweets_vector[p] = 1
            p+=1
        label_list.append(tweets_vector)
        text_list.append(out[0])
numpy_text_list = np.array(text_list)
numpy_label_text_list = np.array(label_list)
np.save('./data/bert_label.npy',numpy_label_list)
np.save('./data/bert_text.npy',numpy_text_list)

12/23/2020 05:52:56 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
12/23/2020 05:52:56 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmp4aa6jcxb
12/23/2020 05:53:00 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size

RuntimeError: ignored